# Neural Network approach with RNN

In [1]:
!python --version

Python 3.7.13


## Libraries and data import

In [2]:
!pip install -U torch==1.10.0
!pip install -U torchtext==0.11.0
!python -m spacy download en_core_web_sm
!pip install -Uqq ipdb

import ipdb

from google.colab import drive
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler
from collections import Counter

import torch
torch.cuda.empty_cache()
import torchtext
from torchtext.legacy import data
from torchtext.legacy.data import Field, Dataset, Example

import random

import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import recall_score, precision_score, fbeta_score

import time

from pathlib import Path  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x3965e000 @  0x7f5ccfbb1615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 19 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.10.0 which is incom

In [3]:
#import of the processed dataset and the columns' names
from_drive = True
dataset = dict()
path = "/content/gdrive/MyDrive/Magistrale/Stage/data"

if from_drive == True: 
  drive.mount("/content/gdrive")
  dataset["ace"] = pd.read_csv(path + "/preprocessed_ace2.csv")
  dataset["copd"] = pd.read_csv(path + "/preprocessed_copd2.csv")
  dataset["ppi"] = pd.read_csv(path + "/preprocessed_ppi2.csv")
else: 
  dataset["ace"] = pd.read_csv("/content/preprocessed_ace2.csv")
  dataset["copd"] = pd.read_csv("/content/preprocessed_copd2.csv")
  dataset["ppi"] = pd.read_csv("/content/preprocessed_ppi2.csv")

Mounted at /content/gdrive


## Data preparation


In [4]:
#random seed for reproducibility
SEED = [1009, 2839, 516, 2383, 273, 1625, 1324, 2791, 7, 1928]
torch.manual_seed(SEED[0])
torch.backends.cudnn.deterministic = True

#FIELD: how the data should be processed
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True) #in order to know how long the actual sequences are
                  #this will cause `batch.text` to now be a tuple with the first element being the sentence 
                  #(a numericalized tensor that has been padded) and the second element being the actual lengths of the sentences

LABEL = data.LabelField(dtype = torch.float)
fields = {'label' : LABEL, "text" : TEXT}

#HYPERPARAMETERS

#dataset configuration
i = "copd" #which dataset
clean_text = True #otherwise, it will be used "text"

#bucket iterator configuration
BATCH_SIZE = 32 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#neural network configuration
EMBEDDING_DIM = 100 #size of the dense word vectors (usually 50-250)
#in this case, must be equal to that of the pre-trained GloVe vectors
HIDDEN_DIM = 256 #size of the hidden states (usually 100-500)
OUTPUT_DIM = 1 #number of classes
N_LAYERS = 5
BIDIRECTIONAL = True
DROPOUT = 0.5

K = 95

To ensure the pre-trained vectors can be loaded into the model, the `EMBEDDING_DIM` must be equal to that of the pre-trained GloVe vectors loaded earlier.

In [5]:
#removal and renaming of columns

if clean_text == True: 
  dataset[i].drop(dataset[i].columns.difference(['Label', "text_clean"]), 1, inplace=True)
  dataset[i].rename(columns={'text_clean': 'text'}, inplace=True)

else: 
  dataset[i].drop(dataset[i].columns.difference(['Label', "text"]), 1, inplace=True)

dataset[i].rename(columns={'Label': 'label'}, inplace=True)

#final dataset with "label" and "text"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [6]:
print(dataset[i].shape)
dataset[i].head()

(1606, 2)


,label,text
0,0,the soluble form of a disintegrin and metallop...
1,1,interactive effect of paraoxonase 1 q192r poly...
2,0,trx ask1 jnk signaling regulation of cell dens...
3,0,increased activation of p38 mapk in copd infla...
4,0,cigarette smoke extract induced cytokine and c...


## Methods definition

In [7]:
class DataFrameDataset(Dataset):
  """Class for using pandas DataFrames as a datasource"""
  def __init__(self, examples, fields, filter_pred=None):
    """
    Create a dataset from a pandas dataframe of examples and Fields
    Arguments:
      examples pd.DataFrame: DataFrame of examples
      fields {str: Field}: The Fields to use in this tuple. The
        string is a field name, and the Field is the associated field.
      filter_pred (callable or None): use only exanples for which
        filter_pred(example) is true, or use all examples if None.
        Default is None
    """
    self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
    if filter_pred is not None:
      self.examples = filter(filter_pred, self.examples)
    self.fields = dict(fields)
    # Unpack field tuples
    for n, f in list(self.fields.items()):
      if isinstance(n, tuple):
        self.fields.update(zip(n, f))
        del self.fields[n]

class SeriesExample(Example):
  """Class to convert a pandas Series to an Example"""

  @classmethod
  def fromSeries(cls, data, fields):
    return cls.fromdict(data.to_dict(), fields)

  @classmethod
  def fromdict(cls, data, fields):
    ex = cls()
      
    for key, field in fields.items():
      if key not in data:
        raise ValueError("Specified key {} was not found in "
        "the input data".format(key))
      if field is not None:
        setattr(ex, key, field.preprocess(data[key]))
      else:
        setattr(ex, key, data[key])
    return ex

**Different RNN Architecture**

Long Short-Term Memory (LSTM) is a type of RNN architecture. 

Standard RNNs suffer from the vanishing gradient problem. LSTMs overcome this by having an extra recurrent state called a _cell_, $c$ - which can be thought of as the "memory" of the LSTM - and using multiple _gates_ which control the flow of information into and out of the memory. We can simply think of the LSTM as a function of $x_t$, $h_t$ and $c_t$, instead of just $x_t$ and $h_t$.

$$(h_t, c_t) = \text{LSTM}(x_t, h_t, c_t)$$

Thus, the model using an LSTM looks something like (with the embedding layers omitted):

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment2.png?raw=1)

The initial cell state, $c_0$, like the initial hidden state is initialized to a tensor of all zeros. The final prediction is still, however, only made using the final hidden state, not the final cell state, i.e. $\hat{y}=f(h_T)$.

\\

**Bidirectional RNN**

The concept behind a bidirectional RNN is simple. As well as having an RNN processing the words in the sentence from the first to the last (a forward RNN), we have a second RNN processing the words in the sentence from the **last to the first** (a backward RNN). At time step $t$, the forward RNN is processing word $x_t$, and the backward RNN is processing word $x_{T-t+1}$. 

In PyTorch, the hidden state (and cell state) tensors returned by the forward and backward RNNs are stacked on top of each other in a single tensor. 

We make our sentiment prediction using a concatenation of the last hidden state from the forward RNN (obtained from final word of the sentence), $h_T^\rightarrow$, and the last hidden state from the backward RNN (obtained from the first word of the sentence), $h_T^\leftarrow$, i.e. $\hat{y}=f(h_T^\rightarrow, h_T^\leftarrow)$   

The image below shows a bi-directional RNN, with the forward RNN in orange, the backward RNN in green and the linear layer in silver.  

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment3.png?raw=1)

\\

**Multi-layer RNN**

Multi-layer RNNs (also called *deep RNNs*) are another simple concept. The idea is that we add additional RNNs on top of the initial standard RNN, where each RNN added is another *layer*. The hidden state output by the first (bottom) RNN at time-step $t$ will be the input to the RNN above it at time step $t$. The prediction is then made from the final hidden state of the final (highest) layer.

The image below shows a multi-layer unidirectional RNN, where the layer number is given as a superscript. Also note that each layer needs their own initial hidden state, $h_0^L$.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment4.png?raw=1)

\\

**Regularization**

Although we've added improvements to our model, each one adds additional parameters. Without going into overfitting into too much detail, the more parameters you have in in your model, the higher the probability that your model will overfit (memorize the training data, causing a low training error but high validation/testing error, i.e. poor generalization to new, unseen examples). To combat this, we use regularization. More specifically, we use a method of regularization called *dropout*. Dropout works by randomly *dropping out* (setting to 0) neurons in a layer during a forward pass. The probability that each neuron is dropped out is set by a hyperparameter and each neuron with dropout applied is considered indepenently. One theory about why dropout works is that a model with parameters dropped out can be seen as a "weaker" (less parameters) model. The predictions from all these "weaker" models (one for each forward pass) get averaged together withinin the parameters of the model. Thus, your one model can be thought of as an ensemble of weaker models, none of which are over-parameterized and thus should not overfit.

In [8]:
class RNN(nn.Module): #the RNN class is a sub-class of nn.Module
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx): #define the layers of the module 
      
    super().__init__()
    
    self.embedding = nn.Embedding(vocab_size, 
                                  embedding_dim, 
                                  padding_idx = pad_idx) #index of the pad token is passed as an argument to the embedding layer
    
    self.rnn = nn.LSTM(embedding_dim, #LSTM layer
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout) #adds dropout on the connections between hidden states in one layer to hidden states in the next layer
    #implementing bidirectionality and adding additional layers are done 
    #by passing values for the num_layers and bidirectional arguments for the RNN/LSTM
    
    self.fc = nn.Linear(hidden_dim * 2, output_dim) #linear layer
    #as the final hidden state of the LSTM has both a forward and a backward component,
    #which will be concatenated together, the size of the input to the nn.Linear layer 
    #is twice that of the hidden dimension size

    self.dropout = nn.Dropout(dropout) #dropout layer
    #the argument is the probability of dropping out each neuron
    #the dropout layer is used within the forward method after each layer we want to apply dropout to

      
  def forward(self, text, text_lengths): #called when feeding examples into the model
  #text_lengths is needed as we are passing the lengths of the sentences 
  #to be able to use packed padded sequences

    #text = [sent len, batch size] (tensor)
    #text is a batch of senteces, each having each word converted into a one-hot vector 

    #the input batch is passed through the embedding layer to get `embedded`, 
    #which gives us a dense vector representation of our sentences.
    embedded = self.dropout(self.embedding(text)) 
    
    #embedded = [sent len, batch size, emb dim] (tensor)
    
    #PACK SEQUENCE: before we pass our embeddings to the RNN, we need to pack them
    #this will cause our RNN to only process the non-padded elements of a sequence
    #lengths need to be on CPU
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
    
    #the RNN will then return packed_output (a packed sequence) as well as the hidden and cell states (both of which are tensors)
    packed_output, (hidden, cell) = self.rnn(packed_embedded) 
    #so LSTM returns the output and a tuple of the final hidden state and the final cell state
    
    #UNPACK SEQUENCE: unpack of the output sequence to transform it from a packed sequence to a tensor
    #the elements of output from padding tokens will be zero tensors (tensors where every element is zero)
    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
    
    #output = [sent len, batch size, hid dim * num directions] (tensor)
    #output is the concatenation of the hidden state from every time step
    #output over padding tokens are zero tensors

    #hidden = [num layers * num directions, batch size, hid dim]
    #cell = [num layers * num directions, batch size, hid dim]
    
    #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
    #and apply dropout
    
    #the layers are ordered: [forward_layer_0, backward_layer_0, forward_layer_1, backward_layer 1, ..., forward_layer_n, backward_layer n]
    #as we want the final layer forward and backward hidden states, 
    #we get the last two hidden layers, hidden[-2,:,:] and hidden[-1,:,:], 
    #and concatenate them together before passing them to the linear layer (after applying dropout)
    hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
            
    #hidden = [batch size, hid dim * num directions]
    
    #the last hidden state, hidden, is fed through the linear layer to produce a prediction
    return self.fc(hidden)

**RNN class notes**: 

- The tensor `text` should have another dimension due to the one-hot vectors, however PyTorch conveniently stores a one-hot vector as it's index value, i.e. the tensor representing a sentence is just a tensor of the indexes for each token in that sentence. The act of converting a list of tokens into a list of indexes is commonly called *numericalizing*.
- Never use dropout on the input or output layers (`text` or `fc` in this case) because you only ever want to use dropout on intermediate layers.
- Without packed padded sequences, `hidden` and `cell` are tensors from the last element in the sequence, which will most probably be a pad token, however when using packed padded sequences they are both from the last non-padded element in the sequence.
- The `lengths` argument of `packed_padded_sequence` must be a CPU tensor so we explicitly make it one by using `.to('cpu')`.
- Usually, there is only the need to unpack output if you are going to use it later on in the model. Although in this case the unpack output is not used, it has been unpacked anyway.

In [9]:
def count_parameters(model):
  """ 
  Function that tells how many trainable parameters the model has 
  so we can compare the number of parameters across different models.
  """
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [10]:
def binary_metrics(preds, y):
  
  """
  Returns metrics per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
  """

  #feed the prediction through a sigmoid layer, squashing the values in [0, 1], then round them to the nearest integer
  rounded_preds = torch.round(torch.sigmoid(preds)) 

  #calculate how many rounded predictions equal the actual labels and average it across the batch
  correct = (rounded_preds == y).float() #convert into float for division 
  acc = correct.sum() / len(correct)

  y_true = y.detach().numpy()
  y_pred = rounded_preds.detach().numpy()

  recall = recall_score(y_true, y_pred, average='binary', zero_division=0)
  precision = precision_score(y_true, y_pred, average='binary', zero_division=0)
  f2 = fbeta_score(y_true, y_pred, average='binary', beta=2, zero_division=0)
  f3 = fbeta_score(y_true, y_pred, average='binary', beta=3, zero_division=0)

  return acc, recall, precision, f2, f3

In [11]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def getTpTnFpFn(df): 

  TP = len(df.loc[df[["target", "prediction"]].eq(1).all(1)])
  TN = len(df.loc[df[["target", "prediction"]].eq(0).all(1)])
  FP = len(df.loc[(df["target"] == 0) & (df["prediction"] == 1)]) 
  FN = df.shape[0] - (TP + TN + FP)

  return TP, TN, FP, FN

def arrange_predictions_and_targets(preds, target): 
  
  #get list of predictions and targets
  predictions = [item.detach().numpy() for sublist in preds for item in sublist]
  targets = [item.detach().numpy() for sublist in target for item in sublist] 

  #create dataframe that contain predictions and targets
  temp = pd.DataFrame()
  temp["prediction_value"] = predictions
  temp["target"] = targets
  #sort in descending order respect to the predictions
  temp.sort_values(by=['prediction_value'], inplace=True, ascending=False, ignore_index=True) 

  #compute the sigmoid function over the predictions (in order to obtain values in [0, 1])
  for index, value in enumerate(temp["prediction_value"]):
    temp.loc[index, "sigmoid_value"] = sigmoid(value)

  #convert predictions from real value to 0 or 1
  temp.loc[temp["sigmoid_value"] >= 0.5, "prediction"] = 1
  temp.loc[temp["sigmoid_value"] < 0.5, "prediction"] = 0
  temp.drop(['prediction_value', "sigmoid_value"], inplace=True, axis=1)

  #cast dataframe type to int
  temp = temp.astype(int)

  return temp

In [12]:
def train_model(model, iterator, optimizer, criterion, epoch, fold, iteration, res): #iterates over all examples, one batch at a time
  
  epoch_loss = 0
  epoch_acc = 0
  epoch_recall = 0
  epoch_precision = 0
  epoch_f2 = 0
  epoch_f3 = 0
  pred = []
  target = []
  
  model.train() #put the model in "training mode", which turns on dropout and batch normalization
  
  for batch in iterator: #for each batch
    
    optimizer.zero_grad() #we first zero the gradients

    #separete batch.text before passing it to the model
    #batch.text is a tuple with the first element being the numericalized tensor 
    #and the second element being the actual lengths of each sequence
    text, text_lengths = batch.text

    #feed the batch of sentences,text, and their lenghts, text_lenghts into the model 
    predictions = model(text, text_lengths).squeeze(1)
    #squeeze is needed as the predictions are initially size [batch size, 1]
    #and we need to remove the dimension of size 1, as PyTorch expects the 
    #predictions input to our criterion function to be of size [batch size]    
    pred.append(predictions.cpu())
    target.append(batch.label.cpu())
    
    #loss and accuracy are then calculated using the predictions and the labels, batch.label, 
    #with the loss being averaged over all examples in the batch    
    
    #computation of loss 
    loss = criterion(predictions, batch.label)
    #criterion expects both input to be FloatTensors
    #that's wht in the label field we set dtype=torch.float 
    
    #computation of accuracy
    acc, recall, precision, f2, f3 = binary_metrics(predictions.cpu(), batch.label.cpu())

    loss.backward() #calculate the gradient of each parameter
    
    optimizer.step() #update the parameters using the gradients and optimizer algorithm

    #the loss and accuracy are accumulated across the epoch, 
    #the .item() method is used to extract a scalar from a tensor which only contains a single value
    epoch_loss += loss.item()
    epoch_acc += acc.item()
    epoch_recall += recall.item()
    epoch_precision += precision.item()
    epoch_f2 += f2.item()
    epoch_f3 += f3.item()

  # OPTION 2

  pred_df = arrange_predictions_and_targets(pred, target)

  #compute TP, TN, FP, FN
  TP, TN, FP, FN = getTpTnFpFn(pred_df)
  retrieved_docs = TP + FP
  relevant_docs = TP + FN
  not_relevant_docs = TN + FP

  #ACCURACY
  acc = (TP + TN) / (TP + TN + FP + FN)

  #PRECISION
  if retrieved_docs == 0: 
    print("number of retrieved docs equal to 0 (in training)")
    precision = 0
  else: 
    precision = TP / retrieved_docs

  #RECALL
  recall = TP / relevant_docs

  #TRUE NEGATIVE RATE
  if not_relevant_docs==0: 
    print("Number of not relevant docs equals to 0 (in training)")
    TNR = 0
  else:
    TNR = TN / not_relevant_docs

  
  #return the loss and accuracy, averaged across the epochs
  #the len of an iterator is the number of batches in the iterator
  metrics = {"df" : i, 
             "set" : "train", 
             "#_fold" : fold,
             "#_iteration" : iteration, 
             "#_epoch" : epoch,
             "loss" : round(epoch_loss / len(iterator), 3), 
             "acc" : round((epoch_acc / len(iterator))*100, 2), 
             "acc2" : round(acc*100, 2),
             "recall" : round((epoch_recall / len(iterator))*100, 2), 
             "recall2" : round(recall*100, 2),
             "precision" : round((epoch_precision / len(iterator))*100, 2), 
             "precision2" : round(recall*100, 2),
             "f2" : round((epoch_f2 / len(iterator))*100, 2), 
             "f3" : round((epoch_f3 / len(iterator))*100, 2), 
             "tnr" : round(TNR*100, 2)}

  return metrics, pred_df

All layers have their parameters initialized to random values, unless explicitly specified.

- **Embedding layer**: is used to transform the sparse one-hot vector into a dense embedding vector (dense as the dimensionality is a lot smaller and all the elements are real numbers). It is simply a single fully connected layer. 
As well as reducing the dimensionality of the input to the RNN, there is the theory that words with similar meaning are mapped close together in this dense vector space.

- **RNN**: takes in the dense vector and the previous hidden state $h_{t-1}$, which it uses to calculate the next hidden state, $h_t$.

- **Linear layer**: takes the final hidden state and feeds it through a fully connected layer, $f(h_T)$, transforming it to the correct output dimension.

**Train_model method notes**: 

- Each parameter in a model has a `grad` attribute which stores the gradient calculated by the `criterion`. PyTorch does not automatically remove (or "zero") the gradients calculated from the last gradient calculation, so they must be manually zeroed.

- You do not need to do `model.forward(batch.text)`, simply calling the model works.

- When initializing the `LABEL` field, we set `dtype=torch.float`. This is because TorchText sets tensors to be `LongTensor`s by default, however our criterion expects both inputs to be `FloatTensor`s. 
The alternative method of doing this would be to do the conversion inside the `train` function by passing `batch.label.float()` instad of `batch.label` to the criterion. 

In [13]:
def evaluate_model(model, iterator, criterion, epoch, fold, iteration, res, set_type): #similar to train, without the the update of the parameters
  
  epoch_loss = 0
  epoch_acc = 0
  epoch_recall = 0
  epoch_precision = 0
  epoch_f2 = 0
  epoch_f3 = 0
  pred = []
  target = []
  
  model.eval() #puts the model in "evaluation mode", which turns off dropout and batch normalization
  
  with torch.no_grad(): #in order to not calculate gradients

    for batch in iterator:

      #separate batch.text
      text, text_lengths = batch.text

      predictions = model(text, text_lengths).squeeze(1)
      pred.append(predictions.cpu())
      target.append(batch.label.cpu())
      
      loss = criterion(predictions, batch.label)
      
      acc, recall, precision, f2, f3 = binary_metrics(predictions.cpu(), batch.label.cpu())
      
      epoch_loss += loss.item()
      epoch_acc += acc.item()
      epoch_recall += recall.item()
      epoch_precision += precision.item()
      epoch_f2 += f2.item()
      epoch_f3 += f3.item()

  # OPTION 2

  pred_df = arrange_predictions_and_targets(pred, target)

  #compute TP, TN, FP, FN
  TP, TN, FP, FN = getTpTnFpFn(pred_df)
  retrieved_docs = TP + FP
  relevant_docs = TP + FN
  not_relevant_docs = TN + FP

  #ACCURACY
  acc = (TP + TN) / (TP + TN + FP + FN)

  #PRECISION
  if retrieved_docs == 0: 
    print("number of retrieved docs equal to 0 (in evaluation)")
    precision = 0
  else: 
    precision = TP / retrieved_docs

  #RECALL
  recall = TP / relevant_docs

  #TRUE NEGATIVE RATE
  if not_relevant_docs==0: 
    print("Number of not relevant docs equals to 0 (in evaluation)")
    TNR = 0
  else:
    TNR = TN / not_relevant_docs

  
  #return the loss and accuracy, averaged across the epochs
  #the len of an iterator is the number of batches in the iterator
  metrics = {"df" : i, 
             "set" : set_type, 
             "#_fold" : fold,
             "#_iteration" : iteration, 
             "#_epoch" : epoch,
             "loss" : round(epoch_loss / len(iterator), 3), 
             "acc" : round((epoch_acc / len(iterator))*100, 2), 
             "acc2" : round(acc*100, 2),
             "recall" : round((epoch_recall / len(iterator))*100, 2), 
             "recall2" : round(recall*100, 2),
             "precision" : round((epoch_precision / len(iterator))*100, 2), 
             "precision2" : round(recall*100, 2),
             "f2" : round((epoch_f2 / len(iterator))*100, 2), 
             "f3" : round((epoch_f3 / len(iterator))*100, 2), 
             "tnr" : round(TNR*100, 2)}


  return metrics, pred_df

**Evaluate_model method notes:**

No gradients are calculated on PyTorch operations inside the `with no_grad()` block. This causes less memory to be used and speeds up computation.

The rest of the function is the same as `train`, with the removal of `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()`, as we do not update the model's parameters when evaluating.

In [14]:
def epoch_time(start_time, end_time):
  """
  Returns how long an epoch takes, in order to compare training times between models
  """
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [15]:
def update_results(res, train, val, test): 
  res = res.append(train, ignore_index=True)
  res = res.append(val, ignore_index=True)
  res = res.append(test, ignore_index=True)
  return res

In [16]:
def compute_avg_std(res): 
  results = pd.DataFrame() 
  results["df"] = ["copd", "copd", "copd"]
  results['set'] = ['train', 'valid', 'test']
  for column in res.columns[5:]: #every column from loss column
    avg_column = column + "_avg"
    std_column = column + "_std"
    results.loc[results["set"] == "train", avg_column] = round(res.loc[res["set"] == "train", column].mean(), 3)
    results.loc[results["set"] == "valid", avg_column] = round(res.loc[res["set"] == "validation", column].mean(), 3)
    results.loc[results["set"] == "test", avg_column] = round(res.loc[res["set"] == "test", column].mean(), 3)
    results.loc[results["set"] == "train", std_column] = round(res.loc[res["set"] == "train", column].std(), 3)
    results.loc[results["set"] == "valid", std_column] = round(res.loc[res["set"] == "validation", column].std(), 3)
    results.loc[results["set"] == "test", std_column] = round(res.loc[res["set"] == "test", column].std(), 3)

  return results

## Model training

In [17]:
res = pd.DataFrame()
best_train_preds_dict = {}
best_valid_preds_dict = {}
test_preds_dict = {}

#cross validation 10 folds
for j in range(len(SEED)): 

  ##### TRAIN-TEST-VAL SPLIT #####

  # TRAIN-TEST SPLIT
  set1, set2 = train_test_split(dataset[i], train_size=0.5, random_state=random.seed(123), shuffle=True, stratify=dataset[i]['label'])

  for k in range(2): #swapping train and test (10x2 cross-validation)

    if k == 0:
        train = set1
        test = set2
    else:
        train = set2
        test = set1

    # OVERSAMPLING ON TRAINING SET

    #instantiating the random over sampler 
    ros = RandomOverSampler(sampling_strategy=0.5, random_state=123)
    #resampling X, y
    X_ros, y_ros = ros.fit_resample(train["text"].values.reshape(-1, 1), train["label"].values)
    #creation of oversampled training set
    oversampled_train = pd.DataFrame(X_ros, columns=["text"])
    oversampled_train["label"] = y_ros

    # PYTORCH DATASET 
    train_data = DataFrameDataset(oversampled_train, fields)
    df = DataFrameDataset(test, fields)
    
    # TEST-VAL SPLIT
    test_data, valid_data = df.split(split_ratio=0.8, stratified=True, strata_field='label', random_state = random.seed(123))

    # CREATION OF VOCABULARY

    #VOCABULARY: look up table where every unique word in your data set has a corresponding _index_ (an integer).
    #each _index_ is used to construct a _one-hot_ vector for each word.
    TEXT.build_vocab(train_data,
                    vectors = "glove.6B.100d", #use of pre-trained word embeddings
                    unk_init = torch.Tensor.normal_) #initialize words in pre-trained embeddings randomly via gaussian distribution

    LABEL.build_vocab(train_data)

    # PRINTS
    if j == 0 and k == 0: 
      #new class distribution 
      print("Label distribution in training set:", Counter(y_ros), "\n")

      print(f'Number of training examples: {len(train_data)}')
      print(f'Number of testing examples: {len(test_data)}')
      print(f'Number of validation examples: {len(valid_data)}\n')

      #checking an example
      print("Training example:", vars(train_data.examples[0]), "\n")

      print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
      print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}\n")

      print("Most common words:", TEXT.vocab.freqs.most_common(20))

      #see the vocabulary directly using either stoi (string to int) or itos (int to string)
      print(TEXT.vocab.itos[:10])

      #check the labels, ensuring 0 is for negative and 1 is for positive
      #LABEL.vocab.stoi = {1:1, 0:0}
      print(LABEL.vocab.stoi)

    # CREATION OF THE ITERATORS

    #ITERATORS: you iterate over iterators in the training/evaluation loop, 
    #and they return a batch of examples (indexed and converted into tensors) at each iteration.

    #BUCKET ITERATORS: special type of iterator that will return a batch of examples 
    #where each example is of a similar length, minimizing the amount of padding per example.

    train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
        (train_data, valid_data, test_data), 
        batch_size = BATCH_SIZE,
        device = device, 
        sort_key = lambda x: len(x.text),
        sort_within_batch = True) #False, #for packed padded sequences all of the tensors within a batch need to be sorted by their lengths

    
    ##### BUILD THE MODEL #####

    # CREATION OF RNN CLASS' INSTANCE

    INPUT_DIM = len(TEXT.vocab) #input dimension: dimension of the one-hot vectors (equal to the vocabulary size)
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] #get the pad token index from the vocabulary
    #this allows to get the actual string representing the pad token from the field's pad_token attribute, which is <pad> by default

    model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, 
                BIDIRECTIONAL, DROPOUT, PAD_IDX)
 
    if j == 0: 
      print(f'The model has {count_parameters(model):,} trainable parameters\n')

    # USE OF WORD EMBEDDING IN THE MODEL 

    #retrieve the embeddings from the field's vocab
    pretrained_embeddings = TEXT.vocab.vectors

    #check the embeddings are the correct size, [vocab size, embedding dim]
    if j == 0:
      print("Pretrained embeddings shape:", pretrained_embeddings.shape, "\n")

    #replace the initial weights of the embedding layer with the pre-trained embeddings
    model.embedding.weight.data.copy_(pretrained_embeddings)

    # INITIALIZATION OF <UNK> AND <PAD> TO ALL ZEROS 

    #get the index of the <unk> tokens
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    #initialize <unk> and <pad> tokens to all zeros
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    #check that the first two rows of the embedding weights matrix have been set to zeros
    #print(model.embedding.weight.data)

    ##### TRAIN THE MODEL #####

    # TRAINING PARAMETERS

    #OPTIMIZER: algorithm we use to update the parameters of the module. 
    #chosen optimizer: Adam
    optimizer = optim.Adam(model.parameters())

    #CRITERION: loss function
    #chosen loss function: binary cross entropy with logits
    weights = [1 - (len(dataset[i][dataset[i]["label"]==0]) / dataset[i].shape[0])]
    weight = torch.FloatTensor(weights)
    criterion = nn.BCEWithLogitsLoss(weight=weight)
    #criterion = nn.BCEWithLogitsLoss()

    #place the model and the criterion on the GPU (if we have one)
    model = model.to(device)
    criterion = criterion.to(device)

    #train the model through multiple epochs
    #EPOCH: complete pass through all examples in the training and validation sets

    N_EPOCHS = 20

    best_val_results = {"loss" : 1}
    
    # TRAINING THE MODEL

    print("\nFOLD", j+1)
    if k == 0: 
        print("FIRST ITERATION\n")
    else: 
        print("SECOND ITERATION\n")

    train_preds_dict = {}
    valid_preds_dict = {}

    for epoch in range(N_EPOCHS):

        start_time = time.time()
        
        #training 
        train_results, train_preds = train_model(model, train_iterator, optimizer, criterion, epoch+1, j+1, k+1, res)
        train_preds_dict.update({epoch+1 : train_preds})
        #evaluation
        val_results, val_preds = evaluate_model(model, valid_iterator, criterion, epoch+1, j+1, k+1, res, "validation")
        valid_preds_dict.update({epoch+1 : val_preds})

        #computation of time
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        #at each epoch, if the validation loss is the best seen so far,
        #we'll save the parameters of the model and then, after training has finished,
        #we'll use that model on the test set
        if val_results["loss"] < best_val_results["loss"]: 
            best_train_results = train_results
            best_val_results = val_results
            torch.save(model.state_dict(), 'tut2-model.pt')
        
        #results for each epoch
        #print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        #print(f'\tTrain Loss: {train_results["loss"]} | Train Acc: {train_results["acc"]}% | Train Recall: {train_results["recall"]}% | Train Precision: {train_results["precision"]}% | Train F2: {train_results["f2"]}% | Train F3: {train_results["f3"]}%')
        #print(f'\t Val. Loss: {val_results["loss"]} |  Val. Acc: {val_results["acc"]}% | Val. Recall: {val_results["recall"]}% | Val. Precision: {val_results["precision"]}% | Val. F2: {val_results["f2"]}% | Val. F3: {val_results["f3"]}%')

    # RESULTS ON TRAIN AND VALIDATION SET WITH THE BEST MODEL

    print(f'Epoch: {best_train_results["#_epoch"]}')
    print(f'\tTrain Loss: {best_train_results["loss"]} | Train Acc: {best_train_results["acc"]}% | Train Recall: {best_train_results["recall"]}% | Train Precision: {best_train_results["precision"]}% | Train F2: {best_train_results["f2"]}% | Train F3: {best_train_results["f3"]}%')
    print(f'\tVal. Loss: {best_val_results["loss"]} |  Val. Acc: {best_val_results["acc"]}% | Val. Recall: {best_val_results["recall"]}% | Val. Precision: {best_val_results["precision"]}% | Val. F2: {best_val_results["f2"]}% | Val. F3: {best_val_results["f3"]}%')

    
    best_train_preds_dict.update({k : train_preds_dict[best_train_results["#_epoch"]]})
    best_valid_preds_dict.update({k : valid_preds_dict[best_train_results["#_epoch"]]})

    # RESULTS ON TEST SET

    #test loss and accuracy (using parameters that gave the best validation loss)
    model.load_state_dict(torch.load('tut2-model.pt'))
    test_results, test_preds = evaluate_model(model, test_iterator, criterion, best_train_results["#_epoch"], j+1, k+1, res, "test")
    print(f'\nTest Loss: {test_results["loss"]} | Test Acc: {test_results["acc"]}% | Test Recall: {test_results["recall"]}% | Test Precision: {test_results["precision"]}% | Test F2: {test_results["f2"]}% | Test F3: {test_results["f3"]}%')

    test_preds_dict.update({k : test_preds})

    res = update_results(res, best_train_results, best_val_results, test_results)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399999/400000 [00:13<00:00, 28778.33it/s]


Label distribution in training set: Counter({0: 705, 1: 352}) 

Number of training examples: 1057
Number of testing examples: 642
Number of validation examples: 161

Training example: {'label': 0, 'text': ['maternal', 'glutathione', 's', 'transferase', 'gstp1', 'genotype', 'is', 'a', 'specific', 'predictor', 'of', 'phenotype', 'in', 'children', 'with', 'asthma', 'maternal', 'factors', 'are', 'known', 'to', 'influence', 'the', 'heritability', 'and', 'expression', 'of', 'asthma', 'and', 'atopy', 'we', 'report', 'the', 'association', 'of', 'maternal', 'paternal', 'and', 'proband', 'gstp1', 'genotype', 'with', 'lung', 'function', 'in', '145', 'caucasian', 'children', 'with', 'asthma', 'gstp1', 'val105', 'val105', 'and', 'ala114', 'val114', 'genotypes', 'in', 'the', 'child', 'were', 'associated', 'with', 'non', 'significant', 'increases', 'in', 'lung', 'function', 'forced', 'expiratory', 'volume', 'in', '1', 'second', 'fev1', 'forced', 'vital', 'capacity', 'fvc', 'and', 'the', 'fev1', 'fvc'

**Notes on "Creation of vocabulary"**

It is important to only build the vocabulary on the training set because, when testing a machine learning system, you do not want to look at the test set in any way. Also, do not include the validation set as you want it to reflect the test set as much as possible.


In the vocabulary there is the addition of the `<pad>` token.

When feeding sentences into the model, you feed a _batch_ of them at a time, i.e. more than one at a time, and all sentences in the batch need to be the same size. Thus, to ensure each sentence in the batch is the same size, any shorter than the longest within the batch are padded.

\\

**Notes on "Creation of RNN class' instance"**

- **Input dimension**: dimension of the one-hot vectors (equal to the vocabulary size).
- **Embedding dimension**: size of the dense word vectors. This is usually around 50-250 dimensions, but depends on the size of the vocabulary.
- **Hidden dimension**: size of the hidden states. This is usually around 100-500 dimensions, but also depends on factors such as on the vocabulary size, the size of the dense vectors and the complexity of the task.
- **Output dimension**: usually the number of classes. However in the case of only 2 classes the output value is between 0 and 1 and thus can be 1-dimensional, i.e. a single scalar real number.

\\

**Notes on "Use of word embedding in the model"**

Replacing the initial weights of the embedding layer with the pre-trained embeddings should always be done on the `weight.data` and not the `weight`.

\\

**Notes on "Initialization of \<unk> and \<pad> to all zeros"**

As our `<unk>` and `<pad>` token aren't in the pre-trained vocabulary they have been initialized using `unk_init` (an $\mathcal{N}(0,1)$ distribution) when building our vocab. It is preferable to initialize them both to all zeros to explicitly tell our model that, initially, they are irrelevant for determining relevance. 

We do this by manually setting their row in the embedding weights matrix to zeros. We get their row by finding the index of the tokens, which we have already done for the padding index.

Like initializing the embeddings, this should be done on the `weight.data` and not the `weight`.

As we passed the index of the pad token to the `padding_idx` of the embedding layer it will remain zeros throughout training, however the `<unk>` token embedding will be learned.

\\

**Notes on "Training parameters"**

**Optimizers** 
- **SGD**: updates all parameters with the same learning rate and choosing this learning rate can be tricky. 
- **Adam**: adapts the learning rate for each parameter, giving parameters that are updated more frequently lower learning rates and parameters that are updated infrequently higher learning rates. 
You do not have to provide an initial learning rate for Adam as PyTorch specifies a sensibile default initial learning rate.

The model currently outputs an unbound real number. As the labels are either 0 or 1, you want to restrict the predictions to a number between 0 and 1. This can be done using the _sigmoid_ or _logit_ functions. 

Is it possible to use the bound scalar to calculate the loss using binary cross entropy. 

The `BCEWithLogitsLoss` criterion carries out both the sigmoid and the binary cross entropy steps.

\\

**Notes on "Training the model"**

We then train the model through multiple epochs, an epoch being a complete pass through all examples in the training and validation sets.

At each epoch, if the validation loss is the best we have seen so far, we'll save the parameters of the model and then after training has finished we'll use that model on the test set.

In [19]:
print(res.shape)
display(res.head())

(60, 15)


,df,set,#_fold,#_iteration,#_epoch,loss,acc,acc2,recall,recall2,precision,precision2,f2,f3,tnr
0,copd,train,1.0,1.0,6.0,0.026,92.46,92.24,83.38,87.78,82.08,87.78,82.00,82.56,94.47
1,copd,validation,1.0,1.0,6.0,0.022,94.79,93.79,52.78,65.00,67.50,65.00,54.74,53.73,97.87
2,copd,test,1.0,1.0,6.0,0.030,92.56,92.21,54.82,58.97,58.83,58.97,53.75,54.15,96.81
3,copd,train,1.0,2.0,17.0,0.014,95.96,95.84,91.78,95.74,86.73,95.74,90.28,90.99,95.89
4,copd,validation,1.0,2.0,17.0,0.019,95.83,95.03,64.72,75.00,71.67,75.00,65.86,65.28,97.87


In [20]:
results = compute_avg_std(res)
display(results.head())

,df,set,loss_avg,loss_std,acc_avg,acc_std,acc2_avg,acc2_std,recall_avg,recall_std,...,precision_avg,precision_std,precision2_avg,precision2_std,f2_avg,f2_std,f3_avg,f3_std,tnr_avg,tnr_std
0,copd,train,0.027,0.013,91.299,4.861,91.045,5.002,83.215,7.240,...,81.556,7.769,87.016,7.668,81.692,7.592,82.353,7.428,93.056,4.645
1,copd,valid,0.024,0.006,93.228,2.383,91.925,2.843,52.970,17.589,...,58.083,13.395,63.750,19.593,52.184,16.238,52.465,16.857,95.921,1.733
2,copd,test,0.032,0.005,91.071,1.825,90.888,1.770,54.672,10.455,...,58.652,8.770,59.230,10.986,53.170,9.330,53.750,9.816,95.267,1.853


## Additional metrics computation on training set

In [21]:
def get_rank_at_k(df, total_relevant_docs): 
  
  relevant_docs = 0
  
  for j in range(len(df)): 
    
    if df.loc[j, "target"] == 1: 
      relevant_docs = relevant_docs + 1
    
    recall_k = relevant_docs / total_relevant_docs
    
    if recall_k >= K/100: 
      return j

  print("Error")
  return 0

In [22]:
def update_results(pred_df, set_type): 

  #compute TP, TN, FP, FN
  TP, TN, FP, FN = getTpTnFpFn(pred_df)
  retrieved_docs = TP + FP
  relevant_docs = TP + FN
  not_relevant_docs = TN + FP

  #EVALUATION MEASURES AT 95% RECALL

  #selection of the rows where recall@95
  last_positive_rank = get_rank_at_k(pred_df, relevant_docs) 
  pred_df_95 = pred_df[:last_positive_rank+1]
  pred_df_5 = pred_df[last_positive_rank+1:]
  TP_95, TN_95, FP_95, FN_95 = getTpTnFpFn(pred_df_95)
  TP_5, TN_5, FP_5, FN_5 = getTpTnFpFn(pred_df_5)

  #TRUE NEGATIVE RATE AT 95% RECALL
  if not_relevant_docs==0: 
    print("Number of not relevant docs equals to 0")
    TNR_k = 0
  else:
    TNR_k = TN_5/not_relevant_docs

  #PRECISION@95
  if retrieved_docs==0: 
    print("Number of retrieved docs equals to 0")
    precision_k = 0
  else:
    precision_k = TP_95/retrieved_docs 

  #WSS@95
  N = len(pred_df)
  WSS_k = ((N - last_positive_rank)/N) - ((100-K)/100)

  #update results dataset
  results.loc[results["set"] == set_type, "true_negative_rate@95"] = round(TNR_k*100, 3)
  results.loc[results["set"] == set_type, "precision@95"] = round(precision_k*100, 3)
  results.loc[results["set"] == set_type, "wss@95"] = round(WSS_k*100, 3)

  print("N:", N)
  print("Last positive rank:", last_positive_rank)
  print()

In [23]:
for j in range(2): 
  update_results(best_train_preds_dict[j], "train")
  update_results(best_valid_preds_dict[j], "valid")
  update_results(test_preds_dict[j], "test")

N: 1057
Last positive rank: 779

N: 161
Last positive rank: 132

N: 642
Last positive rank: 587

N: 1057
Last positive rank: 525

N: 161
Last positive rank: 76

N: 642
Last positive rank: 351



In [24]:
results

,df,set,loss_avg,loss_std,acc_avg,acc_std,acc2_avg,acc2_std,recall_avg,recall_std,...,precision2_std,f2_avg,f2_std,f3_avg,f3_std,tnr_avg,tnr_std,true_negative_rate@95,precision@95,wss@95
0,copd,train,0.027,0.013,91.299,4.861,91.045,5.002,83.215,7.240,...,7.668,81.692,7.592,82.353,7.428,93.056,4.645,72.908,83.429,45.331
1,copd,valid,0.024,0.006,93.228,2.383,91.925,2.843,52.970,17.589,...,19.593,52.184,16.238,52.465,16.857,95.921,1.733,58.865,69.565,47.795
2,copd,test,0.032,0.005,91.071,1.825,90.888,1.770,54.672,10.455,...,10.986,53.170,9.330,53.750,9.816,95.267,1.853,50.887,64.706,40.327


In [25]:
#saving results in a csv file
if from_drive == True: 
  with open(path + "/output_nn_approach.csv", 'w', encoding = 'utf-8-sig') as f:
    res.to_csv(f)
  with open(path + "/output_results_nn_approach.csv", 'w', encoding = 'utf-8-sig') as g:
    results.to_csv(g)  
else: 
  filepath = Path('/content/output_nn_approach.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  res.to_csv(filepath)
  filepath = Path('/content/output_results_nn_approach.csv')  
  filepath.parent.mkdir(parents=True, exist_ok=True)  
  results.to_csv(filepath)